# Testing the Liquidator Agent using crvUSDsim

In [1]:
# Load in a LLAMMA and Controller from crvUSDsim
# Drop the price:
# Update External Market price
# Update LLAMMA price oracle
# TODO other?
# Check if a user can be liquidated
# Perform the liquidation
from crvusdsim.pool import get, SimLLAMMAPool, SimController
from crvusdsim.pool.sim_interface.sim_controller import DEFAULT_LIQUIDATOR
from src.agents.liquidator import Liquidator
from src.sim.scenario import Scenario

%load_ext autoreload
%autoreload 2

In [2]:
liquidator = Liquidator()

ETH_POOL = "weth"
(
    pool,
    controller,
    collateral_token,
    stablecoin,
    aggregator,
    stableswap_pools,
    peg_keepers,
    policy,
    factory,
) = get(ETH_POOL, bands_data="controller")

In [3]:
# Verify snapshot context works
i, j = 1, 0

stableswap_pool = stableswap_pools[2]
high = stableswap_pool.get_max_trade_size(i, j)
print("Balances", stableswap_pool.balances)
print("High", high)
print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])
with stableswap_pool.use_snapshot_context():
    amt_out = stableswap_pool.trade(i, j, high)
    print("Amount out", amt_out)
    print("Balances", stableswap_pool.balances)
    print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])
print("Balances", stableswap_pool.balances)
print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])

Balances [6967972351382, 28733626803122982780380462]
High 10948641974534121542609549
Allowance 6967972351382
Amount out 6897602798605
Balances [70024638146, 39682268777657104322990011]
Allowance 70369552777
Balances [6967972351382, 28733626803122982780380462]
Allowance 6967972351382


In [4]:
config = "../src/configs/scenarios/baseline.json"
scenario = Scenario(config)
markets = scenario.generate_markets()
prices = scenario.generate_pricepaths(
    "../src/configs/prices/1h_1694894242_1700078242.json"
)

[INFO][01:49:54][root]-345299: Reading price config from ../src/configs/scenarios/baseline.json.
[INFO][01:49:54][root]-345299: Fetching 1inch quotes.


[INFO][01:49:57][root]-345299: We have 213120 quotes.
[INFO][01:49:57][root]-345299: Fitting external markets against 1inch quotes.
[INFO][01:49:57][root]-345299: Reading price config from ../src/configs/prices/1h_1694894242_1700078242.json.


In [5]:
# Set External Market Prices
sample = prices[-1].prices
for in_token in markets:
    for out_token in markets[in_token]:
        markets[in_token][out_token].update_price(sample[in_token][out_token])

In [8]:
liquidator.set_paths(controller, stableswap_pools, markets)
# Artificially raise price if necessary to make liquidation profitable
liquidator.paths[0].collat_pool.update_price(5000)

In [9]:
liquidator.perform_liquidations(controller)

[INFO][01:50:09][root]-345299: There are 6 users to liquidate.
[INFO][01:50:09][root]-345299: Liquidating user 0xe3cf28f4c11b6ef4313e7c23df526f75bab0bc64 with expected profit: [13588.30392399].
[INFO][01:50:09][root]-345299: Executing trade Swap(pool=<SimCurveStableSwapPool address=0x4dece678ceceb27446b35c672dc7d61f30bad69e chain=mainnet>, i=0, j=1, amt=13227205926).
[INFO][01:50:09][root]-345299: Executing trade Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7ff3d8df1400>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7ff3d84dc450>, amt=13307109074580921122816, frac=1000000000000000000, i=0, j=1).
[INFO][01:50:09][root]-345299: Executing trade Swap(pool=<src.modules.market.ExternalMarket object at 0x7ff3c70b11d0>, i=0, j=1, amt=5364348417970767872).
[INFO][01:50:09][root]-345299: Liquidated user 0xe3cf28f4c11b6ef4313e7c23df526f75bab0bc64 with profit: [13588.30392399].
[INFO][01:50:09][root]-345299: Liquidating user 0x644

[INFO][01:50:09][root]-345299: Liquidated user 0x408f0c201abf1f0217f16626f963e9a218c9f882 with profit: [17110.93619846].
[INFO][01:50:09][root]-345299: Liquidating user 0x18c0d980a3d271f97f37af443abd495cc30f6900 with expected profit: [15833.97011569].
[INFO][01:50:09][root]-345299: Executing trade Swap(pool=<SimCurveStableSwapPool address=0x4dece678ceceb27446b35c672dc7d61f30bad69e chain=mainnet>, i=0, j=1, amt=49965012656).
[INFO][01:50:09][root]-345299: Executing trade Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7ff3d8df1400>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7ff3d84debd0>, amt=50253141023635575668736, frac=1000000000000000000, i=0, j=1).
[INFO][01:50:09][root]-345299: Executing trade Swap(pool=<src.modules.market.ExternalMarket object at 0x7ff3c70b11d0>, i=0, j=1, amt=13163165124249155584).
[INFO][01:50:09][root]-345299: Liquidated user 0x18c0d980a3d271f97f37af443abd495cc30f6900 with profit: [15833.97011

(array([111018.92454839]), 0)